<a href="https://colab.research.google.com/github/Osmium98/Movie-Recommendation-System/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Block-1

In [2]:
!pip install surprise
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 663.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357274 sha256=4514f164fe84221d95c7c8f4602053add83521f2c49a281bf3f86eacb13fd1db
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


Block-2

In [9]:
import pandas as pd

# Load the datasets
ratings = pd.read_csv('/content/drive/MyDrive/movie_reccomendation_System_CG/ml-latest-small/ratings.csv')
movies = pd.read_csv('/content/drive/MyDrive/movie_reccomendation_System_CG/ml-latest-small/movies.csv')

# Display the first few rows of each dataset
print(ratings.head())
print(movies.head())


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Block-3

In [4]:
# Remove leading/trailing spaces in column names
ratings.columns = [col.strip() for col in ratings.columns]
movies.columns = [col.strip() for col in movies.columns]

# Rename columns to match expected names if they are different
ratings.rename(columns={'UserID': 'userId', 'MovieID': 'movieId', 'Rating': 'rating', 'Timestamp': 'timestamp'}, inplace=True)
movies.rename(columns={'MovieID': 'movieId', 'Title': 'title', 'Genres': 'genres'}, inplace=True)

# Verify column names again
print("Updated Ratings columns:", ratings.columns)
print("Updated Movies columns:", movies.columns)

print(ratings.head())
print(movies.head())


Updated Ratings columns: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Updated Movies columns: Index(['movieId', 'title', 'genres'], dtype='object')
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Block-4

In [5]:
# Convert timestamp to datetime if present
if 'timestamp' in ratings.columns:
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Merge ratings and movies data
data = pd.merge(ratings, movies, on='movieId')

# Display merged data
print(data.head())


   userId  movieId  rating           timestamp             title  \
0       1        1     4.0 2000-07-30 18:45:03  Toy Story (1995)   
1       5        1     4.0 1996-11-08 06:36:02  Toy Story (1995)   
2       7        1     4.5 2005-01-25 06:52:26  Toy Story (1995)   
3      15        1     2.5 2017-11-13 12:59:30  Toy Story (1995)   
4      17        1     4.5 2011-05-18 05:28:03  Toy Story (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1  Adventure|Animation|Children|Comedy|Fantasy  
2  Adventure|Animation|Children|Comedy|Fantasy  
3  Adventure|Animation|Children|Comedy|Fantasy  
4  Adventure|Animation|Children|Comedy|Fantasy  


Block-5

In [6]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate

# Prepare the Surprise dataset
reader = Reader(rating_scale=(0.5, 5.0))
surprise_data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the data
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

# Initialize the SVD model with custom parameters
svd_model = SVD(
    n_factors=100,   # Number of latent factors
    n_epochs=20,     # Number of epochs
    lr_all=0.005,    # Learning rate
    reg_all=0.02     # Regularization term
)

# Train the model
svd_model.fit(trainset)

# Predict and evaluate
svd_predictions = svd_model.test(testset)
from surprise import accuracy
print(f"SVD RMSE: {accuracy.rmse(svd_predictions)}")


RMSE: 0.8816
SVD RMSE: 0.8815798149938084


Block-6

In [22]:
import numpy as np

def extract_features_from_svd(svd_model, df, trainset):
    X = []
    y = []

    for _, row in df.iterrows():
        try:
            uid = trainset.to_inner_uid(row['userId'])
            iid = trainset.to_inner_iid(row['movieId'])
            user_vector = svd_model.pu[uid]
            item_vector = svd_model.qi[iid]
            feature_vector = np.concatenate((user_vector, item_vector))
            X.append(feature_vector)
            y.append(1 if row['rating'] >= 4 else 0)  # Binarize ratings as liked (1) or not liked (0)
        except ValueError:
            # Skip if user or item is not in the training set
            continue

    return np.array(X), np.array(y)

# Extract features for Random Forest
X, y = extract_features_from_svd(svd_model, ratings, trainset)

# Print shapes of X and y to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


X shape: (99969, 200)
y shape: (99969,)


Block-7

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate Random Forest
rf_predictions = rf_model.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, rf_predictions) * 100:.2f}%")


Random Forest Accuracy: 72.13%


Block-8

In [12]:
# Convert SVD predictions for combining
svd_predicted_ratings = [pred.est for pred in svd_predictions]
rf_predicted_ratings = rf_predictions

# # Combine using a simple average
# combined_predictions = (np.array(svd_predicted_ratings) + np.array(rf_predicted_ratings)) / 2
# final_predictions = (combined_predictions >= 0.5).astype(int)

# # Evaluate the combined model
# print(f"Combined Model Accuracy: {accuracy_score(y_test, final_predictions) * 100:.2f}%")


Block-9

In [13]:
# Align predictions by finding the common set of test points
rf_test_indices = np.array([i for i, _ in enumerate(y_test)])
common_indices = np.intersect1d(rf_test_indices, np.array([i for i, _ in enumerate(svd_predictions)]))

# Filter the predictions based on common indices
aligned_svd_predictions = np.array([svd_predicted_ratings[i] for i in common_indices])
aligned_rf_predictions = rf_predicted_ratings[common_indices]

# Combine the aligned predictions
combined_predictions = (aligned_svd_predictions + aligned_rf_predictions) / 2
final_predictions = (combined_predictions >= 0.5).astype(int)

# Evaluate the combined model
print(f"Combined Model Accuracy: {accuracy_score(y_test[common_indices], final_predictions) * 100:.2f}%")


Combined Model Accuracy: 48.05%


Block-10

In [14]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import RandomizedSearchCV, train_test_split

# Prepare the Surprise dataset
reader = Reader(rating_scale=(0.5, 5.0))
surprise_data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Define the parameter distribution
param_dist = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005, 0.01], #add less no.
    'reg_all': [0.02, 0.1]
}

# Perform randomized search
randomized_search = RandomizedSearchCV(SVD, param_distributions=param_dist, n_iter=10, measures=['rmse'], cv=3)
randomized_search.fit(surprise_data)

# Best SVD parameters
best_params = randomized_search.best_params['rmse']
print("Best parameters for SVD:", best_params)

# Train the SVD model with the best parameters
svd_model = SVD(**best_params)

# Split the data
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42) #decrese test size 0.25

# Fit the model
svd_model.fit(trainset)

# Predict and evaluate
svd_predictions = svd_model.test(testset)

from surprise import accuracy
print(f"SVD RMSE: {accuracy.rmse(svd_predictions)}")



Best parameters for SVD: {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1}
RMSE: 0.8614
SVD RMSE: 0.8613536296946086


Block-11

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
rf_param_dist = {
    'n_estimators': [50, 100],# remove 200
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Perform randomized search
rf_randomized_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_distributions=rf_param_dist, n_iter=10, cv=3, scoring='accuracy')
rf_randomized_search.fit(X_train, y_train)

# Best Random Forest parameters
best_rf_params = rf_randomized_search.best_params_
print("Best parameters for Random Forest:", best_rf_params)

# Train the Random Forest model with the best parameters
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)


Best parameters for Random Forest: {'n_estimators': 100, 'min_samples_split': 10, 'max_depth': None}


Block-12

In [20]:
import numpy as np
from sklearn.metrics import accuracy_score

# Debug: Print a sample of X_test to ensure it contains userId and movieId
print("Sample of X_test:")
print(X_test[:5])  # Print the first 5 rows of X_test

# Check if X_test has userId and movieId in the expected positions
if X_test.shape[1] < 2:
    print("Error: X_test does not contain both userId and movieId columns.")
else:
    # Convert SVD predictions into a dictionary for easy lookup
    svd_predicted_dict = {(pred.uid, pred.iid): pred.est for pred in svd_predictions}

    # Initialize lists for aligned predictions
    aligned_svd_predictions = []
    aligned_rf_predictions = []
    aligned_y_test = []

    # Align predictions based on common user-item pairs
    for i, (uid, iid) in enumerate(zip(X_test[:, 0], X_test[:, 1])):  # Assuming X_test has userId and movieId as the first two columns
        try:
            # Check if the user-item pair is present in SVD predictions
            if (uid, iid) in svd_predicted_dict:
                aligned_svd_predictions.append(svd_predicted_dict[(uid, iid)])
                aligned_rf_predictions.append(rf_predictions[i])
                aligned_y_test.append(y_test[i])  # Make sure the test labels are aligned too
            else:
                # Debug: Print which user-item pairs are missing
                print(f"Missing SVD prediction for user {uid} and item {iid}")
        except IndexError:
            # Handle case where index may not exist
            print(f"IndexError: Skipping user {uid} and item {iid}")

    # Convert lists to numpy arrays
    aligned_svd_predictions = np.array(aligned_svd_predictions)
    aligned_rf_predictions = np.array(aligned_rf_predictions)
    aligned_y_test = np.array(aligned_y_test)

    # Debug: Print the lengths of all arrays
    print(f"Aligned SVD Predictions Length: {len(aligned_svd_predictions)}")
    print(f"Aligned RF Predictions Length: {len(aligned_rf_predictions)}")
    print(f"Aligned Test Labels Length: {len(aligned_y_test)}")

    # Combine predictions if lengths match and are non-zero
    if len(aligned_svd_predictions) > 0 and len(aligned_rf_predictions) > 0 and len(aligned_y_test) > 0:
        combined_predictions = (aligned_svd_predictions + aligned_rf_predictions) / 2
        final_predictions = (combined_predictions >= 0.5).astype(int)

        # Evaluate the combined model
        print(f"Combined Model Accuracy: {accuracy_score(aligned_y_test, final_predictions) * 100:.2f}%")
    else:
        print("No common predictions found. Unable to combine and evaluate.")


Streaming output truncated to the last 5000 lines.
Missing SVD prediction for user -0.10614561657229636 and item -0.17391743540871385
Missing SVD prediction for user 0.10075791026440475 and item 0.01719981434864117
Missing SVD prediction for user -0.07838019708011265 and item -0.0388582034297652
Missing SVD prediction for user 0.18025767194978903 and item 0.08356845421667856
Missing SVD prediction for user 0.15087838955908925 and item 0.01110427388224409
Missing SVD prediction for user 0.0101046399029113 and item -0.22195426251339584
Missing SVD prediction for user 0.0101046399029113 and item -0.22195426251339584
Missing SVD prediction for user -0.13059023001290102 and item 0.3756602208096178
Missing SVD prediction for user 0.10899436121659288 and item 0.00737372476844575
Missing SVD prediction for user 0.1548127115369487 and item 0.08134383414968595
Missing SVD prediction for user 0.00042204385859284784 and item 0.12869251914573843
Missing SVD prediction for user -0.16665610411273182 